In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [9]:
# extract Product Title
def get_title(soup):

    try:
        # Find the element containing the full string
        full_text = soup.find("span", attrs={"class": 'base'}).text

        # Split the text at ' - ' and extract the part before it
        product_name = full_text.split(' - ')[0]

    except AttributeError:
        title_string = ""

    return product_name

# Extract Product Processor
def get_processor(soup):
    
    try:
        # Finding the 'Processor Type' label and handle any leading/trailing spaces
          processor_label = soup.find("th", string=lambda text: text and "Processor Type" in text)

        # Finding the next 'td' eement which contains the processor type
          processor = processor_label.find_next("td").text

    except AttributeError:
            processor = ""

    return processor

# Extract Product RAM
def get_ram(soup):
    
    try:
        # Finding the RAM label and handle any leading/trailing spaces
          ram_label = soup.find("th", string=lambda text: text and "Installed RAM" in text)

        # Finding the next 'td' eement which contains the ram type
          ram = ram_label.find_next("td").text

    except AttributeError:
            ram = ""

    return ram

# Extract Product SSD
def get_ssd(soup):
    
    try:
        # Finding the SSD label and handle any leading/trailing spaces
          ssd_label = soup.find("th", string=lambda text: text and "SSD" in text)

        # Finding the next 'td' eement which contains the ssd type
          ssd = ssd_label.find_next("td").text

    except AttributeError:
            ssd = ""

    return ssd

# Function to extract Product touchscreen
def get_touchscreen(soup):
    
    try:
        # Finding the touchscreen label and handle any leading/trailing spaces
          touchscreen_label = soup.find("th", string="Touchscreen")

        # Finding the next 'td' eement which contains the touchscreen type
          touchscreen = touchscreen_label.find_next("td").text

    except AttributeError:
            touchscreen = ""

    return touchscreen

# Extract Product touchscreen
def get_warranty(soup):
    
    try:
        # Find the 'Processor Type' label and handle any leading/trailing spaces
          warranty_label = soup.find("th", string="Warranty")

        # Find the next 'td' eement which contains the touchscreen type
          warranty = warranty_label.find_next("td").text

    except AttributeError:
            warranty = ""

    return warranty

# extract Product Price
def get_price(soup):
 
    try:
        price = soup.find("span", attrs={"class": 'price'}).text

    except AttributeError:
           price = ""

    return price

In [11]:
if __name__ == '__main__':

    # user agent 
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # webpage URL
    URL = "https://www.paklap.pk/laptops-prices/hp-laptops.html"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetching links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'product-item-link'}) 
    
    # Storing the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        href = link.get('href')
        if href:  # Check if the href is not None
            links_list.append(href)

    d = {"Title":[], "Processor":[], "RAM":[], "SSD":[],"Touchscreen":[],"Warranty":[],"Price":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        print(f"Fetching details from: {link}")  # Printing the link being processed
        new_webpage = requests.get(link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['Title'].append(get_title(new_soup))
        d['Processor'].append(get_processor(new_soup))
        d['RAM'].append(get_ram(new_soup))
        d['SSD'].append(get_ssd(new_soup))
        d['Touchscreen'].append(get_touchscreen(new_soup))
        d['Warranty'].append(get_warranty(new_soup))
        d['Price'].append(get_price(new_soup))

    paklap_df = pd.DataFrame.from_dict(d)
    paklap_df['Title'].replace('', np.nan, inplace=True)
    paklap_df = paklap_df.dropna(subset=['Title'])  # Use the correct DataFrame variable name
    paklap_df.to_csv("paklap_data.csv", header=True, index=False)

Fetching details from: https://www.paklap.pk/hp-250-g10-13th-gen-core-i3-laptop-price-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0021nx-13th-generation-core-i3-laptop-price-pakistan.html
Fetching details from: https://www.paklap.pk/hp-250-g10-13th-gen-ci3-laptop-price-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fq5292nia-12th-gen-core-i5-laptop-pakistan.html
Fetching details from: https://www.paklap.pk/hp-250-g9-12th-generation-core-i5-laptop-price-reviews-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0211nia-13th-gen-core-i5-laptop-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0311tu-13th-gen-core-i5-laptop-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0355nia-13th-gen-core-i5-laptop-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0240tu-13th-gen-core-i5-laptop-pakistan.html
Fetching details from: https://www.paklap.pk/hp-15-fd0346nia-13th-generation-core-i7-l

In [12]:
paklap_df

,Title,Processor,RAM,SSD,Touchscreen,Warranty,Price
0,HP 250 G10,13th Generation Intel Core i3-1315U Raptor Lak...,4 GB,256 GB SSD,No,International,"Rs. 105,500.00"
1,HP 15 FD0021nx,13th Generation Intel Core i3-1315U Raptor Lak...,4 GB,256 GB SSD,,International,"Rs. 105,500.00"
2,HP 250 G10,13th Generation Intel Core i3-1315U Raptor Lak...,8 GB,512 GB SSD,No,International,"Rs. 117,500.00"
3,HP 15s FQ5292nia,12th Generation Intel Core i5-1235u Processor ...,08 GB,512 GB SSD,No,International,"Rs. 136,500.00"
4,HP 250 G9,12th Generation Intel Core i5-1235u Processor ...,16 GB,512 GB SSD,No,International,"Rs. 145,500.00"
5,HP 15 FD0211nia,13th Gen Core i5 1335u Raptor Lake Processor (...,8-GB,512 GB SSD,No,International,"Rs. 146,500.00"
6,HP 15 FD0311tu,13th Gen Core i5 1334u Raptor Lake Processor (...,8 GB,512 GB SSD,No,International,"Rs. 147,500.00"
7,HP 15 FD0355nia,13th Gen Core i5 1334u Raptor Lake Processor (...,8 GB,512 GB SSD,No,International,"Rs. 147,900.00"
8,HP 15 FD0240TU,13th Gen Core i5 1334u Raptor Lake Processor (...,8-GB,512 GB SSD,No,1 Year Local,"Rs. 154,500.00"
9,HP 15 FD0346nia,13th Gen Core i7 1355u Raptor Lake Processor (...,8 GB,512 GB SSD,No,International,"Rs. 177,500.00"
